In [1]:
import os
import sys

os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["PYSPARK_PYTHON"]        = sys.executable



In [2]:
from pyspark import SparkContext
from sympy import *
from drudge import *
from gristmill import *
print("finished importing")

finished importing


In [3]:
import re

# ----------------------------------------------------------------------------
# 1) Start Spark & BCS quasiparticle drudge
# ----------------------------------------------------------------------------
ctx = SparkContext('local[*]', 'bcs_ccsd')
dr  = ReducedBCSDrudge(ctx)
dr.full_simplify = True
print("finished")


26/01/19 16:57:08 WARN Utils: Your hostname, Swarnamoys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.99.78.14 instead (on interface en0)
26/01/19 16:57:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 16:57:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

finished


In [4]:
p, q, r, s, i, j, k, l = dr.names.A_dumms[:8]
#p,q,r = dr.dumms('p q r')
u, v = IndexedBase('u'), IndexedBase('v')
E0 = Symbol(r'E_0')
H120, H110 , H021 = IndexedBase('H120'), IndexedBase('H110'), IndexedBase('H021')
H011 , H111 = IndexedBase('H011'), IndexedBase('H111')
H002, H200 = IndexedBase('H002'), IndexedBase('H200')
H100 = IndexedBase('H100') 
H101, H010 = IndexedBase('H101'), IndexedBase('H010')
H001, H020 = IndexedBase('H001'), IndexedBase('H020')
H003, H210 ,H012, H300, H030 = IndexedBase('H003'), IndexedBase('H210'), IndexedBase('H012'), IndexedBase('H300'), IndexedBase('H030')
H102 = IndexedBase('H102')
H201 = IndexedBase('H201')
H000 = IndexedBase('H000')
# Quartic (full set)
H400 = IndexedBase('H400')
H040 = IndexedBase('H040')
H004 = IndexedBase('H004')

H310 = IndexedBase('H310')
H301 = IndexedBase('H301')
H130 = IndexedBase('H130')
H031 = IndexedBase('H031')
H103 = IndexedBase('H103')
H013 = IndexedBase('H013')

H220 = IndexedBase('H220')
H202 = IndexedBase('H202')
H022 = IndexedBase('H022')

H211 = IndexedBase('H211')
H121 = IndexedBase('H121')
H112 = IndexedBase('H112')


In [5]:
P, Pdag, N = dr.names.P, dr.names.Pdag, dr.names.N

In [ ]:
expr_can = H120[p,q,r]*Pdag[p]*N[q]*N[r] + H110[p,q]*Pdag[p]*N[q] +H021[p,q,r]*N[p]*N[q]*P[r]+ H011[p,q]*N[p]*P[q]\
 + H100[p]*Pdag[p] + H001[p]*P[p] + H010[p]*N[p] + H020[p,q]*N[p]*N[q]
H_N_can =  dr.einst(expr_can).normal_order().simplify().merge()
H_N_can.display()

In [6]:
expr = H120[p,q,r]*Pdag[p]*N[q]*N[r] + H110[p,q]*Pdag[p]*N[q] +H021[p,q,r]*N[p]*N[q]*P[r]+ H011[p,q]*N[p]*P[q]\
 + H100[p]*Pdag[p] + H001[p]*P[p] + H010[p]*N[p] + H020[p,q]*N[p]*N[q] + H300[p,q,r]*Pdag[p]*Pdag[q]*Pdag[r]\
+ H030[p,q,r]*N[p]*N[q]*N[r] + H003[p,q,r]*P[p]*P[q]*P[r] +H101[p,q]*Pdag[p]*P[q] + H002[p,q]*P[p]*P[q] + H210[p,q,r]*Pdag[p]*Pdag[q]*N[r] + H200[p,q]*Pdag[p]*Pdag[q]\
+ H111[p,q,r]*Pdag[p]*N[q]*P[r] + H012[p,q,r]*N[p]*P[q]*P[r] + H102[p,q,r]*Pdag[p]*P[q]*P[r] + H201[p,q,r]*Pdag[p]*Pdag[q]*P[r]
 

In [7]:
 
expr4 = (
    H040[p,q,r,s] * N[p]*N[q]*N[r]*N[s] + H130[p,q,r,s] * Pdag[p]*N[q]*N[r]*N[s]
  + H220[p,q,r,s] * Pdag[p]*Pdag[q]*N[r]*N[s] + H310[p,q,r,s] * Pdag[p]*Pdag[q]*Pdag[r]*N[s]
  + H121[p,q,r,s] * Pdag[p]*N[q]*N[r]*P[s] + H022[p,q,r,s] * N[p]*N[q]*P[r]*P[s]
  + H211[p,q,r,s] * Pdag[p]*Pdag[q]*N[r]*P[s]
  + H112[p,q,r,s] * Pdag[p]*N[q]*P[r]*P[s]
  + H013[p,q,r,s] * N[p]*P[q]*P[r]*P[s]
  + H400[p,q,r,s] * Pdag[p]*Pdag[q]*Pdag[r]*Pdag[s]
  + H301[p,q,r,s] * Pdag[p]*Pdag[q]*Pdag[r]*P[s]
  + H202[p,q,r,s] * Pdag[p]*Pdag[q]*P[r]*P[s]
  + H103[p,q,r,s] * Pdag[p]*P[q]*P[r]*P[s]
  + H004[p,q,r,s] * P[p]*P[q]*P[r]*P[s]
  + H031[p,q,r,s] * N[p]*N[q]*N[r]*P[s]
)

Expr = expr + expr4
H_N =  dr.einst(Expr).normal_order().simplify().merge()
H_N.display()

<IPython.core.display.Math object>

In [8]:
print(len(H_N.local_terms))

34


In [ ]:
'''for i, term in enumerate(H_N.local_terms):
    print(f"--- Term {i} ---")
       # The scalar coefficient
    print("Operators (vecs):", term.vecs)   # The associated vector operators
    print()'''


In [9]:
#t1, t2, t3 = IndexedBase('t1'), IndexedBase('t2'), IndexedBase('t3')

t1 = IndexedBase('t1')
t2 = IndexedBase('t2')
t3 = IndexedBase('t3')
t4 = IndexedBase('t4')

T1 = dr.einst(t1[p] * Pdag[p])
T2 = dr.einst(t2[p,q] *Pdag[p]*Pdag[q])/2
T3 = dr.einst(Rational(1,6)*t3[p,q,r]*Pdag[p]*Pdag[q]*Pdag[r])
T4 = dr.einst(t4[p,q,r,s]*Pdag[p]*Pdag[q]*Pdag[r]*Pdag[s])/24
cluster =  T1+T2
cluster.display()

<IPython.core.display.Math object>

In [10]:
dr.set_symm(t2, Perm([1, 0],IDENT), valence=2)   # Enforce t[p,q] = t[q,p]
dr.set_symm(H002,Perm([1,0],IDENT),valence=2)
dr.set_symm(H200,Perm([1,0],IDENT),valence=2)
dr.set_symm(H020, Perm([1, 0],IDENT), valence=2)
dr.set_symm(H021, Perm([1, 0,2],IDENT), valence=3) #This is correct, the symmetry factor is already 
dr.set_symm(H030,Perm([1,0,2], IDENT), Perm([0,2,1], IDENT),Perm([2,1,0], IDENT),valence=3)
dr.set_symm(H300,Perm([1,0,2], IDENT), Perm([0,2,1], IDENT),Perm([2,1,0], IDENT),valence=3)
dr.set_symm(H003,Perm([1,0,2], IDENT), Perm([0,2,1], IDENT),Perm([2,1,0], IDENT),valence=3)
dr.set_symm(H102, Perm([0,2,1], IDENT))
dr.set_symm(H210, Perm([1,0,2], IDENT))
dr.set_symm(H012, Perm([0,2,1], IDENT))
dr.set_symm(H030, Perm([1,0,2], IDENT), Perm([0,2,1], IDENT), valence=3)
dr.set_symm(H300, Perm([1,0,2], IDENT), Perm([0,2,1], IDENT), valence=3)
dr.set_symm(H003, Perm([1,0,2], IDENT), Perm([0,2,1], IDENT), valence=3)
dr.set_symm(H040,Perm([1,0,2,3], IDENT), Perm([0,2,1,3], IDENT), Perm([0,1,3,2], IDENT),valence=4)
dr.set_symm(H130,Perm([0,2,1,3], IDENT), Perm([0,1,3,2], IDENT),valence=4)
dr.set_symm(H220, Perm([1,0,2,3], IDENT),Perm([0,1,3,2], IDENT), valence=4)
dr.set_symm(H310,Perm([1,0,2,3], IDENT),Perm([0,2,1,3], IDENT), valence=4)
dr.set_symm(H121, Perm([0,2,1,3], IDENT), valence=4)
dr.set_symm(H022,Perm([1,0,2,3], IDENT),Perm([0,1,3,2], IDENT),valence=4)
dr.set_symm(H211,Perm([1,0,2,3], IDENT),valence=4)
dr.set_symm(H112,Perm([0,1,3,2], IDENT), valence=4)
dr.set_symm(H013,Perm([0,2,1,3], IDENT), Perm([0,1,3,2], IDENT),  # swap r,s
            valence=4)
dr.set_symm(H400,Perm([1,0,2,3], IDENT),Perm([0,2,1,3], IDENT),Perm([0,1,3,2], IDENT),valence=4)
dr.set_symm(H004,Perm([1,0,2,3], IDENT),Perm([0,2,1,3], IDENT),Perm([0,1,3,2], IDENT),valence=4)
dr.set_symm(H301,Perm([1,0,2,3], IDENT),Perm([0,2,1,3], IDENT),valence=4)
dr.set_symm(H202,Perm([1,0,2,3], IDENT),Perm([0,1,3,2], IDENT),valence=4)
dr.set_symm(H103,Perm([0,2,1,3], IDENT),Perm([0,1,3,2], IDENT),valence=4)
dr.set_symm(H031,Perm([1,0,2,3], IDENT),Perm([0,2,1,3], IDENT),valence=4)












In [11]:
zero_term = [
    (H200[p,p],0),
    (H002[p,p],0),
    (H003[p,p,p],0),
    (H300[p,p,p],0),
    (t2[p,p],0),
    (t3[p,p,q],0),
    (t3[q,p,p],0),
    (t3[p,q,p],0),
    (t4[p,p,r,s],0),
    (t4[r,p,p,s],0),
    (t4[r,s,p,p],0),
    (t4[p,r,p,s],0),
    (t4[p,r,s,p],0),
    (t4[r,p,s,p],0),
]

# --- Quartic H blocks: Pauli / hard-core pair constraints ---

zero_term += [
    # H400[p,q,r,s] * Pdag[p] Pdag[q] Pdag[r] Pdag[s]
    (H400[p,p,r,s], 0),
    (H400[p,r,p,s], 0),
    (H400[p,r,s,p], 0),
    (H400[r,p,p,s], 0),
    (H400[r,p,s,p], 0),
    (H400[r,s,p,p], 0),

    # H004[p,q,r,s] * P[p] P[q] P[r] P[s]
    (H004[p,p,r,s], 0),
    (H004[p,r,p,s], 0),
    (H004[p,r,s,p], 0),
    (H004[r,p,p,s], 0),
    (H004[r,p,s,p], 0),
    (H004[r,s,p,p], 0),

    # H310[p,q,r,s] * Pdag[p] Pdag[q] Pdag[r] N[s]  (no repeats among p,q,r)
    (H310[p,p,r,s], 0),
    (H310[p,r,p,s], 0),
    (H310[r,p,p,s], 0),

    # H301[p,q,r,s] * Pdag[p] Pdag[q] Pdag[r] P[s]  (no repeats among p,q,r)
    (H301[p,p,r,s], 0),
    (H301[p,r,p,s], 0),
    (H301[r,p,p,s], 0),

    # H220[p,q,r,s] * Pdag[p] Pdag[q] N[r] N[s]  (no repeats among p,q)
    (H220[p,p,r,s], 0),

    # H211[p,q,r,s] * Pdag[p] Pdag[q] N[r] P[s]  (no repeats among p,q)
    (H211[p,p,r,s], 0),

    # H202[p,q,r,s] * Pdag[p] Pdag[q] P[r] P[s]
    #   - no repeats among (p,q)
    #   - no repeats among (r,s)
    (H202[p,p,r,s], 0),
    (H202[p,q,r,r], 0),

    # H112[p,q,r,s] * Pdag[p] N[q] P[r] P[s]  (no repeats among r,s)
    (H112[p,q,r,r], 0),

    # H022[p,q,r,s] * N[p] N[q] P[r] P[s]  (no repeats among r,s)
    (H022[p,q,r,r], 0),

    # H103[p,q,r,s] * Pdag[p] P[q] P[r] P[s]  (no repeats among q,r,s)
    (H103[p,q,q,s], 0),
    (H103[p,q,r,q], 0),
    (H103[p,q,r,r], 0),

    # H013[p,q,r,s] * N[p] P[q] P[r] P[s]  (no repeats among q,r,s)
    (H013[p,q,q,s], 0),
    (H013[p,q,r,q], 0),
    (H013[p,q,r,r], 0),
]


In [12]:
Hbar = H_N
curr = H_N
fact = 1
for n in range(8):                              # up to 4-fold for 2-body H
    comm = (curr | cluster).simplify()          # ad_T^{n+1}(H_N)
    fact *= (n + 1)                             # 1!,2!,3!,4!
    Hbar += comm / fact                         # add 1/(n+1)! term
    curr = comm    
print("Hbar evaluation done")

[Stage 297:=======================>                                (5 + 7) / 12]

Hbar evaluation done


In [13]:
E_eqn  = Hbar.simplify().eval_vev().simplify()
E_eqn.display()

<IPython.core.display.Math object>

In [14]:
#E_eqn_CCD = E_eqn.subst(t2[q,q],0).simplify().cache()
#E_eqn_CCD = E_eqn.subst(t2[p,p],0).simplify().cache()
E_eqn_CCSD = E_eqn.subst_all(zero_term)
print(E_eqn_CCSD.latex())

\sum_{p \in A} \sum_{q \in A} H^{(002)}_{p,q}  t^{(2)}_{p,q}   + \sum_{p \in A} \sum_{q \in A} t^{(1)}_{p}  t^{(1)}_{q}  H^{(002)}_{p,q}  - \sum_{p \in A} \sum_{q \in A} 6 t^{(1)}_{p}  t^{(2)}_{p,q}  H^{(003)}_{p,p,q}  - \sum_{p \in A} \sum_{q \in A} 3 t^{(1)}_{p}^{2}  t^{(1)}_{q}  H^{(003)}_{p,p,q}   + \sum_{p \in A} H^{(001)}_{p}  t^{(1)}_{p}   + \sum_{p \in A} \sum_{q \in A} \sum_{r \in A} \sum_{s \in A} 3 t^{(2)}_{p,s}  t^{(2)}_{q,r}  H^{(004)}_{p,q,r,s}   + \sum_{p \in A} \sum_{q \in A} \sum_{r \in A} \sum_{s \in A} 6 t^{(1)}_{p}  t^{(1)}_{q}  t^{(2)}_{r,s}  H^{(004)}_{p,q,r,s}   + \sum_{p \in A} \sum_{q \in A} \sum_{r \in A} \sum_{s \in A} t^{(1)}_{p}  t^{(1)}_{q}  t^{(1)}_{r}  t^{(1)}_{s}  H^{(004)}_{p,q,r,s}   + \sum_{p \in A} \sum_{q \in A} \sum_{r \in A} 3 t^{(1)}_{p}  t^{(2)}_{q,r}  H^{(003)}_{p,q,r}   + \sum_{p \in A} \sum_{q \in A} \sum_{r \in A} t^{(1)}_{p}  t^{(1)}_{q}  t^{(1)}_{r}  H^{(003)}_{p,q,r} 


In [ ]:
s1_eqn_CCSD = (P[p] * Hbar).simplify().eval_vev().simplify()
print('evaluation done')

In [ ]:
s1_eqn_CCSD = s1_eqn_CCSD.subst_all(zero_term)
#print(s1_eqn_CCSD.latex())

In [ ]:
s2_eqn_CCSD = (P[p]*P[q] * Hbar).simplify().eval_vev().simplify()
s2_eqn_CCSD = s2_eqn_CCSD.subst_all(zero_term)
print('evaluation done')

In [ ]:
print((s2_eqn_CCSD.n_terms))

In [ ]:
def drop_delta_terms_str(tens: Tensor) -> Tensor:
    # Remove any term whose printed form contains 'KroneckerDelta('
    filt = tens.terms.filter(lambda term: 'KroneckerDelta' not in str(term))
    return Tensor(tens._drudge, filt).merge().simplify()

s1_eqn_CCSD = drop_delta_terms_str(s1_eqn_CCSD)
s2_eqn_CCSD = drop_delta_terms_str(s2_eqn_CCSD)

In [ ]:
print((s2_eqn_CCSD.latex()))

In [ ]:
#from drudge import Tensor
#from gristmill import FortranPrinter, optimize, get_flop_cost
#from sympy import IndexedBase

 

Ene = IndexedBase('Ene')
expr0 = dr.define(Ene,E_eqn_CCSD)
Res1 = IndexedBase('Res1')
expr1 = dr.define(Res1[p],s1_eqn_CCSD)
Res2 = IndexedBase('Res2')
expr2 = dr.define(Res2[p,q],s2_eqn_CCSD)
 

eval_equ0 = optimize(
    [expr0,expr1,expr2],
    interm_fmt='tau{}',drop_cutoff=2
)
print("Original cost:", get_flop_cost(eval_equ0))

fort = FortranPrinter()
print("Optimized cost:", get_flop_cost(eval_equ0))
 
evals = fort.doprint(eval_equ0)
with open('SFS_BCS_CCSD.f90', 'w') as fp:
    fp.write(evals)

print("Printed!")

 


 

 

